# STEP ONE: DATA ACQUISITION AND MANIPULATION

In [1]:
# Importing the necessary libraries
import pandas as pd
import datetime as dt

### OVERVIEW OF COLUMNS IN THE "GOONS" SHEET
| COLUMN | DESCRIPTION | EXAMPLE |
| ----------- | ----------- | ------- |
| ATHLETE | The athlete to whom the activity belongs. | "PATRICK L" |
| ACTIVITY ID | The identifier (ID) of the activity extracted. | 8963381312 |
| RUN | The name (i.e. title). | "WarmUp+3x1M MP/1'R + Strides" |        
| MOVING TIME | The moving time (HH:MM:SS). | "00:48:05" |
| DISTANCE | The distance (00.00). | 5.75 |
| PACE | The pace (HH:MM:SS). | "00:08:30" |        
| FULL DATE | The date (MM/DD/YYYY). | "4/26/2023" |        
| TIME | The time (HH:MM:SS AM/PM). | "6:10:12 PM" |        
| DAY | The weekday. | "WED" |        
| MONTH | The month. | 4 |        
| DATE | The day of the month. | 26 |        
| YEAR | The year. | 2023 |        
| SPM AVG | The average number of strides per minute (i.e. cadence). | 167 |        
| HR AVG | The average heart rate (in beats/minute). | 145.4 |        
| WKT TYPE | The run type (0 = None, 1 = Race, 2 = Long run, 3 = Workout). | 3 |        
| DESCRIPTION | The description, or caption. | "Bitch it ain't even my birthday, but I can ball if I want to." |        
| TOTAL ELEV GAIN | The total elevation gain (in meters). | 77.5 |        
| MANUAL | Whether the run was manual or not. | "FALSE" |
| MAX SPEED | The highest speed (in meters/second). | 5.171 |                
| CALORIES | The number of calories burned. | 622 |                
| ACHIEVEMENT COUNT | The number of achievements gained (e.g. Strava PRs). | 0 |                
| KUDOS COUNT | The number of kudos received. | 3 |                
| COMMENT COUNT | The number of comments received. | 3 |                
| ATHLETE COUNT | The number of (identified) athletes run with. | 1 |                                

In [2]:
# Reading in the CSV file and displaying the first five activities
activities_df = pd.read_csv("C:/Users/17178/Desktop/GITHUB_PROJECTS/Strava-API-and-Sheets-Integration/python/data/GOONS_ACTIVITIES.csv", header=0)
activities_df.head(5)

,ATHLETE,ACTIVITY ID,RUN,MOVING TIME,DISTANCE,PACE,FULL DATE,TIME,DAY,MONTH,...,WKT TYPE,DESCRIPTION,TOTAL ELEV GAIN,MANUAL,MAX SPEED,CALORIES,ACHIEVEMENT COUNT,KUDOS COUNT,COMMENT COUNT,ATHLETE COUNT
0,ME,10772716827,Morning Run,00:45:00,6.20,00:07:15,2/16/2024,6:48:56 AM,FRI,2,...,NaN,NaN,101.0,False,4.660,714.0,0.0,9.0,0.0,1.0
1,PATRICK L,10772471929,Easy Run,00:55:04,5.02,00:11:05,2/16/2024,5:57:14 AM,FRI,2,...,0.0,Post: Stretch and Mobility,0.0,False,2.900,523.0,0.0,6.0,0.0,1.0
2,KARTIK K,10771086626,"PLRTD42B: 4 x 200, 4 x Broken-Up 2Ks, 2 x (300...",00:37:03,7.00,00:05:22,2/15/2024,9:30:00 PM,THU,2,...,3.0,"(38, 38, 37, 34). (6:31 as 2:38, 36, 2:39. 38)...",0.0,True,0.000,847.6,0.0,1.0,0.0,1.0
3,KARTIK K,10771053103,PLRTD42A: Warmup and Cooldown,00:27:03,4.00,00:06:53,2/15/2024,8:30:00 PM,THU,2,...,0.0,4 miles of warmup and cooldown - WU was 2 mile...,0.0,True,0.000,484.3,0.0,0.0,0.0,1.0
4,MARK M,10770490860,Evening Run,01:07:01,8.23,00:08:10,2/15/2024,6:57:04 PM,THU,2,...,0.0,"1600,2x8,4x4,8x2 2’ between,90” in \n5:23,2:40...",0.0,False,9.436,0.0,0.0,15.0,0.0,1.0


In [3]:
print(activities_df["WKT TYPE"].unique())

[nan  0.  3.  2.  1.]


In [4]:
# Converting WKT TYPE to be an int
activities_df.loc[activities_df["WKT TYPE"].isna() | activities_df["WKT TYPE"].isnull(), "WKT TYPE"] = activities_df["WKT TYPE"].mask(activities_df["WKT TYPE"].isna() | activities_df["WKT TYPE"].isnull(), "0")
activities_df["WKT TYPE"] = activities_df["WKT TYPE"].astype("int").astype("category")

# Converting full date to a sortable format
activities_df["FULL DATE"] = pd.to_datetime(activities_df["FULL DATE"], yearfirst=False, dayfirst=False).dt.date

# Converting moving time to a sortable (and arithmetically-friendly) format
activities_df['MOVING TIME'] = pd.to_datetime(activities_df['MOVING TIME'], format='%H:%M:%S').dt.time

# Converting time to a sortable (24h) format (for new column, "24H TIME")
activities_df["24H TIME"] = pd.to_datetime(activities_df["TIME"], format='%I:%M:%S %p').dt.time

# Sorting the dataframe by full date and time in descending order
activities_df = activities_df.sort_values(by=["FULL DATE", "24H TIME"], ascending=[False, False]) # Extra precaution (things should be sorted properly as is)

In [5]:
activities_df.head(3)

,ATHLETE,ACTIVITY ID,RUN,MOVING TIME,DISTANCE,PACE,FULL DATE,TIME,DAY,MONTH,...,DESCRIPTION,TOTAL ELEV GAIN,MANUAL,MAX SPEED,CALORIES,ACHIEVEMENT COUNT,KUDOS COUNT,COMMENT COUNT,ATHLETE COUNT,24H TIME
0,ME,10772716827,Morning Run,00:45:00,6.20,00:07:15,2024-02-16,6:48:56 AM,FRI,2,...,NaN,101.0,False,4.66,714.0,0.0,9.0,0.0,1.0,06:48:56
1,PATRICK L,10772471929,Easy Run,00:55:04,5.02,00:11:05,2024-02-16,5:57:14 AM,FRI,2,...,Post: Stretch and Mobility,0.0,False,2.90,523.0,0.0,6.0,0.0,1.0,05:57:14
2,KARTIK K,10771086626,"PLRTD42B: 4 x 200, 4 x Broken-Up 2Ks, 2 x (300...",00:37:03,7.00,00:05:22,2024-02-15,9:30:00 PM,THU,2,...,"(38, 38, 37, 34). (6:31 as 2:38, 36, 2:39. 38)...",0.0,True,0.00,847.6,0.0,1.0,0.0,1.0,21:30:00


In [6]:
# Viewing data types of each column
activities_df.dtypes

ATHLETE                object
ACTIVITY ID             int64
RUN                    object
MOVING TIME            object
DISTANCE              float64
PACE                   object
FULL DATE              object
TIME                   object
DAY                    object
MONTH                   int64
DATE                    int64
YEAR                    int64
SPM AVG               float64
HR AVG                float64
WKT TYPE             category
DESCRIPTION            object
TOTAL ELEV GAIN       float64
MANUAL                 object
MAX SPEED             float64
CALORIES              float64
ACHIEVEMENT COUNT     float64
KUDOS COUNT           float64
COMMENT COUNT         float64
ATHLETE COUNT         float64
24H TIME               object
dtype: object

### INITIAL DATA INSIGHTS

In [9]:
# TODO: Gather interesting, higher-level basic insights on each athlete (e.g. getting average pace graph)
df_pat = activities_df[activities_df["ATHLETE"] == "PATRICK L"]
df_pat = df_pat[df_pat["YEAR"] >= 2021]
df_pat[["SPM AVG", "HR AVG", "MAX SPEED"]].mean().round(2) # Just getting averages for some basic numerical columns

SPM AVG      163.97
HR AVG       138.18
MAX SPEED      4.08
dtype: float64

### MACHINE LEARNING

In [8]:
# TODO: 
# 1) Clean out unhelpful/inappropriate data (determine what that means)
        # We might not want data that has NA values (NA HR, MANUAL RUNS, ETC.) - This could mean losing an entire set of data for an athlete (for HR).
            # Certainly omit the manual runs, if any exist.
        # We should probably omit warmup and cooldown runs from workout days for those who do that.
            # Detecting this would involve checking the runs done on a given day for a given athlete and seeing if there were multiple around the same time.
                # If that's the case, and WKT TYPE is 3 (better yet), we can be confident that the shorter runs (1st and 3rd) are WU and CD and omit such activities.
        # Do we want to require HR, or should we just ignore that altogether? I think we could gain some good insights from it.
        # We can probably omit the following columns:
            # ACTIVITY ID (irrelevant)
            # RUN (uninterpretable string)
            # DAY, MONTH, DATE (if needed, we can extract such fields from FULL DATE)
            # TIME (uninterpretable unlike 24H TIME)
            # DESCRIPTION (uninterpretable string)
            # THE FOUR COUNT VARIABLES (we could keep them, I'd be interested to see if they play a part in training)
                # Theory: It could help athletes to stay motivated. That is, receiving kudos, comments, having people to run with, and achievements.
                # Caveat: The counts could be inaccurate and out-of-date. An activity could be extracted not long after being posted,
                    # in which case the counts would be lower than if it were extracted later. We'd essentially have to update these for a lot of the rows.
# 2) Add more columns per the google doc (for this ML) and do any further cleaning as necessary
    # Days run from last X days, mileage from last X days, time from last X days, average HR from last X days, ...
# 3) Think through what columns would be useful here and create a new DataFrame with only those valuable (and interpretable) predictors
# 4) Test out some ML modeling on what we got and make any necessary revisions
    # We're shooting for model 2 for now
# 5) Finalize a model